# Hi and welcome to my IBM Capstone Week 3 Assignment

I'm going to try and talk my way through my process as I go so that I can capture what I was doing/thinking at the time. These markdown cells should describe the process going on in the code cells below

---

## Part 1.

Firstly I am going to need to install beautifulsoup4 as I haven't used this yet in my environment. 

In [1]:
#conda install -c conda-forge beautifulsoup4

Now I will need to import the required dependencies for the webscraping element of this assignment.

In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

print('Libraries imported.')

Libraries imported.


Ok, now I need to get the data from the Wikipedia page on the Toronto Postal codes. 

We have been given the website url so I will add that to a url variable. 
Then I will use request to get the full html text of that url and BeautifulSoup with the lxml parser to give it to me in a form I can use. 

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

Having looked at the website through the Chrome inspector I know that the data I need sits within a table with a class of 'wikitable sortable' so I will try and pull out the table. 

I'm going to parse through all the table tags for rows in the table, these are labelled as 'tr' tags. Within the 'tr' tags are the 'td' tags that contain the information we need. I will find all of those from each row and assign to a list called cell. The cell 0th item will be the postcode, the 1st will be the borough and the 2nd the neighborhood. (For some reason the neighborhood cell has multiple lines so I need to strip out the text). 

I then want to add the results into 3 lists, one for each column, however I also need to concatenate the neighborhoods when the Postal Code is a repeat. I do this by looking up the last added postcode to the list, and if the current one matches, then add a comma and the new neighborhood into the neighborhood list.

In [4]:
# Get the table from the webpage
table = soup.find('table', class_='wikitable sortable')
table = table.tbody

# initialise lists
pc_list = []
bor_list = []
hood_list = []

# loop through the rows in the table
for row in table.find_all('tr'):
    # get the row as an indexed list
    cell = row.find_all('td')
    # the first row is headers and doesnt have 'td' so skip this row
    if len(cell) > 0:
        # extract the cell contents to variables
        postcode = cell[0].find(text=True)
        borough = cell[1].find(text=True)
        neighborhood = cell[2].find(text=True).strip()
        
        # check that the list has some values otherwise there in an index error
        if len(pc_list) < 1:
            pc_list.append(postcode)
            bor_list.append(borough)
            hood_list.append(neighborhood)
        # see if the postcode matches the last entered value in the list
        elif postcode == str(pc_list[-1]):
            # update just the neighborhood list with the added values
            hood_list[-1] = hood_list[-1] + ', ' + neighborhood
        # otherwise add a new row to all the lists
        else:
            pc_list.append(postcode)
            bor_list.append(borough)
            hood_list.append(neighborhood)


So we should now have 3 list of equal length extracted from the table. 

In [5]:
print(len(pc_list), len(bor_list), len(hood_list))

180 180 180


I can put these lists into a dataframe and start the process of cleaning and transforming to get the data in the final format that I need. 

Firstly I will change the "Not assigned" to None so that I can find and drop those rows easily later.

In [6]:
df_Toronto = pd.DataFrame()
df_Toronto['PostalCode'] = pc_list
df_Toronto['Borough'] = bor_list
df_Toronto['Neighborhood'] = hood_list
df_Toronto = df_Toronto.replace({'Not assigned': None})
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,None,None
1,M2A,None,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"


Now I need to drop the rows where the Borough column is None, I will also need to reset the index of the dataframe

In [7]:
df_Toronto = df_Toronto[df_Toronto['Borough'].notnull()]
df_Toronto.reset_index(drop=True, inplace=True)
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,None


One final thing to do is to make sure that any Null() values in the dataframe are fixed. This would be anytime the Borough and Neighborhood are the same, the Borough is filled out but the Neighborhood is left blank. I need to make the Neighborhood match the Borough.

In [8]:
df_new = df_Toronto[df_Toronto['Neighborhood'].isnull()]
print(df_new)

  PostalCode       Borough Neighborhood
4        M7A  Queen's Park         None


It looks like there is only one place this happens, I can correct in the original dataframe

In [9]:
df_Toronto.loc[4, 'Neighborhood'] = df_Toronto.loc[4,'Borough']
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


The last request in this section is to show the number of rows and column in my completed dataframe. 

In [10]:
df_Toronto.shape

(103, 3)

---

## Part 2.

We now need to get the latitude and longitude coordinates for the centres of each of the Boroughs. First I will install the geocoder package as I haven't used it here before.

In [11]:
#conda install -c conda-forge geocoder

In [21]:
#import geocoder # import geocoder
# postcode = 'M5G'

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
#     lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

# print(latitude, longitude)


KeyboardInterrupt: 

The above code snippet took **way** too long to resolve and that was just one single postcode, so I downloaded the csv file provided in the project directions. 

I now need to convert the csv into a dataframe and merge it with the original `df_toronto` dataframe. The name of the first column doesn't quite match with the other dataframe so I will change it. 


In [30]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')

df_coords.rename(columns={
    'Postal Code': 'PostalCode'},
    inplace=True)

df_coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
df_toronto_merged = pd.merge(df_Toronto, df_coords, on='PostalCode')
df_toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
